In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import folium
import math

# Clustering Accidents with [DBSCAN](https://en.wikipedia.org/wiki/DBSCAN)
Clustering method for mulicipalities report (phase 0).

DBSCAN has 2 mandetory parameters. I suggest using:
- min_samples=2; This means that every 2 points that are close enough to each other can form a cluster. Since road accidents usually happen along a road or in the intersection of two roads, clusters of accidents will usually form a line (not necessarily straight) or some kind of T or X shape.
- eps=?; Parameter still needs to be chosen wisely. When min_samples==2, the eps parameter determines how close two points need to be for them to be included in the same cluster

# Read CSV (commented out - how to download from google drive)

In [2]:
## Code to read csv file from google drive into Colaboratory:
#!pip install -U -q PyDrive
#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#
## Authenticate and create the PyDrive client.
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)
#
#link = '' # add the link to your csv here!
#fluff, id = link.split('=')
#downloaded = drive.CreateFile({'id':id}) 
#downloaded.GetContentFile('tlv_area_involved_markers_walkers.csv')  
df = pd.read_csv('tlv_area_involved_markers_walkers.csv')

## if running with google colab - follow the instructions that appear bellow:

# Some data organization and investigation:

In [3]:
df.shape

(37690, 137)

In [4]:
columns = ['accident_id','longitude','latitude','x','y','injury_severity', 'sex', 
           'age_group', 'home_region','day_night','day_in_week',
          'accident_district','accident_natural_area','accident_yishuv_shape',
           'street1','street1_hebrew','street2','street2_hebrew','accident_year',
          'accident_month','accident_day','accident_hour',
          'location_accuracy','road_type','cross_mode','cross_location','accident_severity']

tlvfrom2015 = df.loc[(df.accident_year >= 2015) & (df.involved_type == 3) 
                          & (df.longitude < 34.81) & (df.latitude > 32.03)][columns]

In [5]:
tlvfrom2015.shape

(2868, 27)

In [6]:
tlvfrom2015.location_accuracy.value_counts(dropna=False)

1    2084
3     679
2     105
Name: location_accuracy, dtype: int64

In [7]:
tlvfrom2015[tlvfrom2015.location_accuracy == 1].accident_severity.value_counts(dropna=False)

3    1814
2     232
1      38
Name: accident_severity, dtype: int64

# Clusters calculation with DBSCAN

In [8]:
X = tlvfrom2015[tlvfrom2015.location_accuracy == 1][['accident_id','x','y','longitude','latitude']]
X.drop_duplicates(subset = 'accident_id',keep = 'first', inplace = True)
db = DBSCAN(eps=50, min_samples=2).fit(X[['x','y']].values)
unique_elements, counts_elements = np.unique(db.labels_, return_counts=True)

In [9]:
# print('unique_elements: ' , unique_elements[counts_elements > 6])
# print('counts_elements: ' , counts_elements[counts_elements > 6])

# Show clusters with largest amount of points on map

In [10]:
m = folium.Map(location=[32.09, 34.78], zoom_start=15)

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
unique_labels = set(labels)

colors = ['#7FFF00','#FFFF00','#FF8C00','#FF0000']

for k in unique_labels:
    class_member_mask = (labels == k)
    xy = X[['longitude','latitude']].values[class_member_mask]
    radius = 3
    cluster_size = len(xy)
    
    if k == -1 or (cluster_size <= 6):
      # Black used for noise.
      col='#000000'
      radius = 1
    else:
      col = colors[min([cluster_size - 7,3])]
      
    for row in xy:
        folium.CircleMarker(location = [row[1], row[0]],radius=radius,color=col,
                            fill_color=col,fill_opacity=0.7).add_to(m)

      
severe = tlvfrom2015[(tlvfrom2015.location_accuracy == 1) & 
                          (tlvfrom2015.accident_severity == 1)]
medium = tlvfrom2015[(tlvfrom2015.location_accuracy == 1) & 
                          (tlvfrom2015.accident_severity == 2)]

# show map:
display(m)

# List locations of data points related to largest clusters

In [11]:
for k in unique_labels:    
    class_member_mask = (labels == k)
    xy = X[['longitude','latitude']].values[class_member_mask]    
    
    cluster_size = len(xy)
    
    if k == -1 or (cluster_size <= 6):
      # Black used for noise.
      pass
    else:
      print('number of events: ' + str(cluster_size))
      print(xy)


number of events: 15
[[34.77384824 32.08106063]
 [34.77385799 32.08124103]
 [34.77390521 32.08020413]
 [34.77393885 32.07980746]
 [34.77393885 32.07980746]
 [34.77393885 32.07980746]
 [34.77386014 32.08078112]
 [34.77393885 32.07980746]
 [34.77388302 32.08042048]
 [34.77386056 32.08069094]
 [34.77392775 32.07991563]
 [34.77393885 32.07980746]
 [34.77393885 32.07980746]
 [34.77387196 32.08051964]
 [34.77384866 32.08097046]]
number of events: 8
[[34.76851139 32.08075381]
 [34.76851139 32.08075381]
 [34.76851139 32.08075381]
 [34.76830312 32.08000461]
 [34.7684285  32.08036576]
 [34.76830312 32.08000461]
 [34.7684897  32.08086195]
 [34.768468   32.08097009]]
number of events: 7
[[34.78373795 32.04704221]
 [34.78433103 32.04700811]
 [34.78373795 32.04704221]
 [34.78375962 32.04693407]
 [34.78386488 32.04706969]
 [34.7837908  32.04706042]
 [34.78387551 32.0470607 ]]
number of events: 8
[[34.7761779  32.05618813]
 [34.7761779  32.05618813]
 [34.7761779  32.05618813]
 [34.7761779  32.05618813